In [1]:
import pandas as p
import numpy as n
import matplotlib.pyplot as plt

In [2]:
data=p.read_csv('NY property data.csv')
data.head()

,RECORD,BBLE,BORO,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


In [3]:
r=data['OWNER'].value_counts().head(20).index.tolist()
r

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'YORKVILLE TOWERS ASSO',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 '434 M LLC',
 'LINCOLN PLAZA ASSOCIA',
 'DEUTSCHE BANK NATIONA',
 '561 11TH AVENUE TMG L']

In [4]:
r.remove('YORKVILLE TOWERS ASSO')
r.remove('434 M LLC')
r.remove('DEUTSCHE BANK NATIONA')
r.remove('561 11TH AVENUE TMG L')
r.append('U S GOVERNMENT OWNRD')
r.append('UNITED STATES OF AMER')
r.append('THE CITY OF NEW YORK')
r.append('NYS URBAN DEVELOPMENT')
r.append('NYS DEPT OF ENVIRONME')
r.append('CULTURAL AFFAIRS')
r.append('DEPT OF GENERAL SERVI')
r.append('DEPT RE_CITY OF NY')

In [5]:
r

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 'LINCOLN PLAZA ASSOCIA',
 'U S GOVERNMENT OWNRD',
 'UNITED STATES OF AMER',
 'THE CITY OF NEW YORK',
 'NYS URBAN DEVELOPMENT',
 'NYS DEPT OF ENVIRONME',
 'CULTURAL AFFAIRS',
 'DEPT OF GENERAL SERVI',
 'DEPT RE_CITY OF NY']

In [6]:
r.remove('DEPT RE_CITY OF NY')
r.append('DEPT RE-CITY OF NY')

In [7]:
sample=data[~data['OWNER'].isin(r)].reset_index(drop=True)

In [8]:
sample.shape

(1046826, 32)

### Missing value imputation

#### ZIP

In [9]:
m=n.where(p.isnull(sample['ZIP']))[0]
len(m)

21772

In [10]:
for i in range(len(m)):
    sample.loc[m[i],'ZIP']=sample.loc[m[i]-1,'ZIP']
        

In [11]:
m=n.where(p.isnull(sample['ZIP']))[0]
len(m)

0

#### STORIES

In [12]:
t=round(sample.groupby('TAXCLASS')['STORIES'].mean(),2)
t

TAXCLASS
1      2.11
1A     1.66
1B     4.00
1C     3.05
1D     1.07
2     16.31
2A     2.84
2B     4.00
2C     4.74
3      1.00
4      5.52
Name: STORIES, dtype: float64

In [13]:
sample['STORIES']=sample['STORIES'].fillna(value=0)

In [14]:
(sample['STORIES']==0).sum()

43968

In [15]:
for i in t.index:
    sample.loc[(sample['STORIES']==0)&(sample['TAXCLASS']==i),'STORIES']=t[i]

In [16]:
sample['STORIES']

0           5.52
1           5.52
2          50.00
3          50.00
4          50.00
           ...  
1046821     2.50
1046822     2.50
1046823     2.00
1046824     2.00
1046825     2.00
Name: STORIES, Length: 1046826, dtype: float64

In [17]:
(sample['STORIES']==0).sum()

0

#### FULLVAL, AVLAND, AVTOT


In [18]:
sample['FULLVAL'].isna().sum()

0

In [19]:
temp=sample[sample['FULLVAL']!=0]
f=temp.groupby('TAXCLASS')['FULLVAL'].mean()

In [20]:
temp=sample[sample['AVLAND']!=0]
a=temp.groupby('TAXCLASS')['AVLAND'].mean()

In [21]:
temp=sample[sample['AVTOT']!=0]
avt=temp.groupby('TAXCLASS')['AVTOT'].mean()

In [22]:
sample['FULLVAL']=sample['FULLVAL'].fillna(value=0)
sample['AVLAND']=sample['AVLAND'].fillna(value=0)
sample['AVTOT']=sample['AVTOT'].fillna(value=0)

In [23]:
for i in f.index:
    sample.loc[(sample['FULLVAL']==0)&(sample['TAXCLASS']==i),'FULLVAL']=f[i]
    sample.loc[(sample['AVLAND']==0)&(sample['TAXCLASS']==i),'AVLAND']=a[i]
    sample.loc[(sample['AVTOT']==0)&(sample['TAXCLASS']==i),'AVTOT']=avt[i]

#### LOT AND BUILDING SIZE

In [24]:
z=['LTFRONT','LTDEPTH','BLDFRONT','BLDDEPTH']
for i in z:
    sample.loc[sample[i]==0,i]=n.nan
    sample.loc[sample[i]==1,i]=n.nan
    

In [25]:
ltf=sample.groupby('TAXCLASS')['LTFRONT'].mean()
ltd=sample.groupby('TAXCLASS')['LTDEPTH'].mean()
bldf=sample.groupby('TAXCLASS')['BLDFRONT'].mean()
bldd=sample.groupby('TAXCLASS')['BLDDEPTH'].mean()

In [26]:
for i in ltf.index:
    sample.loc[(sample['LTFRONT'].isnull())&(sample['TAXCLASS']==i),'LTFRONT']=ltf[i]
    sample.loc[(sample['LTDEPTH'].isnull())&(sample['TAXCLASS']==i),'LTDEPTH']=ltd[i]
    sample.loc[(sample['BLDFRONT'].isnull())&(sample['TAXCLASS']==i),'BLDFRONT']=bldf[i]
    sample.loc[(sample['BLDDEPTH'].isnull())&(sample['TAXCLASS']==i),'BLDDEPTH']=bldd[i]
    

### Variable creation

In [27]:
sample['ZIP']=sample['ZIP'].astype(str)
sample['zip3']=sample['ZIP'].str[:3]

In [28]:
sample['ltsize']=sample['LTFRONT']*sample['LTDEPTH']
sample['bldsize']=sample['BLDFRONT']*sample['BLDDEPTH']
sample['bldvol']=sample['bldsize']*sample['STORIES']

In [29]:
sample['r1']=sample['FULLVAL']/sample['ltsize']
sample['r2']=sample['FULLVAL']/sample['bldsize']
sample['r3']=sample['FULLVAL']/sample['bldvol']
sample['r4']=sample['AVLAND']/sample['ltsize']
sample['r5']=sample['AVLAND']/sample['bldsize']
sample['r6']=sample['AVLAND']/sample['bldvol']
sample['r7']=sample['AVTOT']/sample['ltsize']
sample['r8']=sample['AVTOT']/sample['bldsize']
sample['r9']=sample['AVTOT']/sample['bldvol']

In [30]:
#sample.head().transpose()

In [31]:
var=['r1','r2','r3','r4','r5','r6','r7','r8','r9']
zipm=sample.groupby('ZIP')[var].mean()
zip3m=sample.groupby('zip3')[var].mean()
taxm=sample.groupby('TAXCLASS')[var].mean()
borom=sample.groupby('BORO')[var].mean()

In [32]:
zipm

,r1,r2,r3,r4,r5,r6,r7,r8,r9
ZIP,,,,,,,,,
10001.0,549.795206,716.883268,136.981718,82.637842,109.740911,26.540190,209.814732,268.984579,47.934000
10002.0,352.125998,448.615898,107.921442,40.427291,50.976960,12.851370,110.406623,139.095791,30.521118
10003.0,588.887449,797.964200,178.196071,44.231332,59.227425,13.429318,131.456157,171.798366,31.681222
10004.0,426.099319,501.101842,43.690030,56.095364,69.534410,5.918050,178.639520,211.827721,16.574632
10005.0,130.391459,151.288260,9.069608,20.180111,22.832907,1.587186,57.196453,66.453250,3.754636
...,...,...,...,...,...,...,...,...,...
11694.0,141.422130,605.907054,315.606540,5.881496,20.047830,10.725202,10.995764,37.184449,19.661131
11695.0,108.191259,12929.163544,6681.666779,31.701018,2923.159956,1013.570465,33.058478,3285.953190,1213.457449
11696.0,30.178792,46.934346,8.502599,13.580456,21.120456,3.826170,13.580456,21.120456,3.826170


In [33]:
sample=sample.join(zipm,on='ZIP',rsuffix='_zip5')
print(sample)
sample=sample.join(zip3m,on='zip3',rsuffix='_zip3')
sample=sample.join(taxm,on='TAXCLASS',rsuffix='_taxclass')
sample=sample.join(borom,on='BORO',rsuffix='_boro')

rsuffix=['_zip5','_zip3','_taxclass','_boro']
for v in var:
    for r in rsuffix:
        sample[str(v)+r]=sample[v]/sample[str(v)+r]

          RECORD        BBLE  BORO  BLOCK   LOT EASEMENT  \
0              1  1000010101     1      1   101      NaN   
1              2  1000010201     1      1   201      NaN   
2              9  1000041001     1      4  1001      NaN   
3             10  1000041002     1      4  1002      NaN   
4             11  1000041003     1      4  1003      NaN   
...          ...         ...   ...    ...   ...      ...   
1046821  1070990  5080500083     5   8050    83      NaN   
1046822  1070991  5080500086     5   8050    86      NaN   
1046823  1070992  5080500089     5   8050    89      NaN   
1046824  1070993  5080500092     5   8050    92      NaN   
1046825  1070994  5080500094     5   8050    94      NaN   

                         OWNER BLDGCL TAXCLASS     LTFRONT  ...           r9  \
0        U S GOVT LAND & BLDGS     P7        4  500.000000  ...   325.631450   
1        U S GOVT LAND & BLDGS     Z9        4   27.000000  ...  2948.942760   
2            TRZ HOLDINGS, LLC     R5  

In [34]:
sample.head().transpose()

,0,1,2,3,4
RECORD,1,2,9,10,11
BBLE,1000010101,1000010201,1000041001,1000041002,1000041003
BORO,1,1,1,1,1
BLOCK,1,1,4,4,4
LOT,101,201,1001,1002,1003
...,...,...,...,...,...
r5_boro,16.606462,56.239136,2.499883,3.612808,3.802332
r6_boro,13.679021,46.325119,0.227335,0.328543,0.345778
r7_boro,0.198288,279.513149,1.866605,2.697598,5.015767
r8_boro,14.042014,127.165526,2.58843,3.740773,6.95539


In [35]:
sample.columns

Index(['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3', 'ltsize', 'bldsize',
       'bldvol', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9',
       'r1_zip5', 'r2_zip5', 'r3_zip5', 'r4_zip5', 'r5_zip5', 'r6_zip5',
       'r7_zip5', 'r8_zip5', 'r9_zip5', 'r1_zip3', 'r2_zip3', 'r3_zip3',
       'r4_zip3', 'r5_zip3', 'r6_zip3', 'r7_zip3', 'r8_zip3', 'r9_zip3',
       'r1_taxclass', 'r2_taxclass', 'r3_taxclass', 'r4_taxclass',
       'r5_taxclass', 'r6_taxclass', 'r7_taxclass', 'r8_taxclass',
       'r9_taxclass', 'r1_boro', 'r2_boro', 'r3_boro', 'r4_boro', 'r5_boro',
       'r6_boro', 'r7_boro', 'r8_boro', 'r9_boro'],
      dtype='object')

In [36]:
sample2=sample.copy()


In [37]:
sample.drop(['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3', 'ltsize', 'bldsize',
       'bldvol'],axis=1)

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r1_zip5,...,r9_taxclass,r1_boro,r2_boro,r3_boro,r4_boro,r5_boro,r6_boro,r7_boro,r8_boro,r9_boro
0,40.917782,3994.412456,723.625445,8.079350,788.709805,142.882211,18.413002,1797.485605,325.631450,0.096029,...,4.713325,0.113902,7.712077,6.466016,0.241395,16.606462,13.679021,0.198288,14.042014,14.950034
1,57678.882190,36173.697854,6553.206133,4258.951518,2671.030012,483.882249,25955.496986,16278.164034,2948.942760,135.364878,...,42.684227,160.560217,69.841147,58.556725,127.249178,56.239136,46.325119,279.513149,127.165526,135.388630
2,385.182796,736.308714,14.726174,62.110738,118.729804,2.374596,173.332268,331.338940,6.626779,0.903974,...,0.095919,1.072230,1.421603,0.131587,1.855748,2.499883,0.227335,1.866605,2.588430,0.304241
3,556.662267,1064.105879,21.282118,89.761892,171.587267,3.431745,250.498069,478.847739,9.576955,1.306414,...,0.138621,1.549576,2.054487,0.190168,2.681911,3.612808,0.328543,2.697598,3.740773,0.439687
4,1035.027624,1978.540749,39.570815,94.470682,180.588507,3.611770,465.762431,890.343337,17.806867,2.429076,...,0.257744,2.881198,3.820001,0.353588,2.822600,3.802332,0.345778,5.015767,6.955390,0.817529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046821,60.833333,561.538462,224.615385,2.018889,18.635897,7.454359,3.239167,29.900000,11.960000,0.423922,...,0.824085,0.398834,1.200624,0.879509,0.226228,1.008640,0.738335,0.150641,0.942288,0.685266
1046822,38.483305,653.846154,261.538462,0.658107,11.181490,4.472596,1.056098,17.943510,7.177404,0.268174,...,0.494548,0.252303,1.397986,1.024086,0.073745,0.605181,0.442999,0.049115,0.565484,0.411240
1046823,33.174373,625.000000,312.500000,1.022185,19.257812,9.628906,1.587878,29.915365,14.957682,0.231178,...,1.030636,0.217497,1.336310,1.223633,0.114542,1.042300,0.953718,0.073846,0.942772,0.857022
1046824,76.781674,808.035714,404.017857,1.751697,18.434524,9.217262,3.257494,34.281250,17.140625,0.535059,...,1.181048,0.503394,1.727658,1.581982,0.196288,0.997741,0.912946,0.151494,1.080362,0.982097


In [38]:
x=['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3', 'ltsize', 'bldsize',
       'bldvol']
for i in x:
    sample=sample.drop(i,axis=1)

In [39]:
sample

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r1_zip5,...,r9_taxclass,r1_boro,r2_boro,r3_boro,r4_boro,r5_boro,r6_boro,r7_boro,r8_boro,r9_boro
0,40.917782,3994.412456,723.625445,8.079350,788.709805,142.882211,18.413002,1797.485605,325.631450,0.096029,...,4.713325,0.113902,7.712077,6.466016,0.241395,16.606462,13.679021,0.198288,14.042014,14.950034
1,57678.882190,36173.697854,6553.206133,4258.951518,2671.030012,483.882249,25955.496986,16278.164034,2948.942760,135.364878,...,42.684227,160.560217,69.841147,58.556725,127.249178,56.239136,46.325119,279.513149,127.165526,135.388630
2,385.182796,736.308714,14.726174,62.110738,118.729804,2.374596,173.332268,331.338940,6.626779,0.903974,...,0.095919,1.072230,1.421603,0.131587,1.855748,2.499883,0.227335,1.866605,2.588430,0.304241
3,556.662267,1064.105879,21.282118,89.761892,171.587267,3.431745,250.498069,478.847739,9.576955,1.306414,...,0.138621,1.549576,2.054487,0.190168,2.681911,3.612808,0.328543,2.697598,3.740773,0.439687
4,1035.027624,1978.540749,39.570815,94.470682,180.588507,3.611770,465.762431,890.343337,17.806867,2.429076,...,0.257744,2.881198,3.820001,0.353588,2.822600,3.802332,0.345778,5.015767,6.955390,0.817529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046821,60.833333,561.538462,224.615385,2.018889,18.635897,7.454359,3.239167,29.900000,11.960000,0.423922,...,0.824085,0.398834,1.200624,0.879509,0.226228,1.008640,0.738335,0.150641,0.942288,0.685266
1046822,38.483305,653.846154,261.538462,0.658107,11.181490,4.472596,1.056098,17.943510,7.177404,0.268174,...,0.494548,0.252303,1.397986,1.024086,0.073745,0.605181,0.442999,0.049115,0.565484,0.411240
1046823,33.174373,625.000000,312.500000,1.022185,19.257812,9.628906,1.587878,29.915365,14.957682,0.231178,...,1.030636,0.217497,1.336310,1.223633,0.114542,1.042300,0.953718,0.073846,0.942772,0.857022
1046824,76.781674,808.035714,404.017857,1.751697,18.434524,9.217262,3.257494,34.281250,17.140625,0.535059,...,1.181048,0.503394,1.727658,1.581982,0.196288,0.997741,0.912946,0.151494,1.080362,0.982097


In [40]:
output=sample.describe()

In [41]:
output=round(output,2)

In [42]:
output.transpose()

,count,mean,std,min,25%,50%,75%,max
r1,1046826.0,213.57,532.56,0.0,76.26,152.86,243.18,138637.34
r2,1046826.0,536.42,1027.44,0.0,211.82,500.00,683.82,310551.57
r3,1046826.0,244.08,498.82,0.0,82.50,239.06,336.23,227500.00
r4,1046826.0,10.46,69.98,0.0,2.35,4.60,7.23,22249.80
r5,1046826.0,22.82,378.51,0.0,6.20,14.98,20.29,334636.37
r6,1046826.0,10.04,178.33,0.0,2.02,7.15,10.04,111545.46
r7,1046826.0,26.83,203.87,0.0,5.28,8.58,13.53,64711.85
r8,1046826.0,50.46,932.20,0.0,17.47,26.98,36.00,871362.22
r9,1046826.0,19.07,327.02,0.0,7.09,12.68,17.58,290454.07
r1_zip5,1046826.0,1.00,2.16,0.0,0.46,0.92,1.23,747.76


In [43]:
output.transpose().to_excel('stats_on_vars.xlsx')